Тестовое задание на Junior CV Engineer

Главное — чтобы задачи решались, и сам процесс был полезным (а ещё лучше — интересным) для всех. Поэтому предлагаем два варианта проекта:

1. Проект по классификации наличия святого Георгия на изображении. В папке находятся два файла со списками изображений: "Георгиев" и "не Георгиев". Необходимо создать Jupyter Notebook, в котором будет обучаться модель для классификации изображений по этим двум категориям. Скачать файлы можно с помощью команды `wget --random-wait -i filename.txt`.  


In [ ]:
pip install wget

In [ ]:
from pathlib import Path
import pandas as pd
import os
import wget
from sklearn.model_selection import train_test_split
import tensorflow as tf
from PIL import Image, ImageOps
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

Определяем пути

In [ ]:
general_path =Path('/content/drive/MyDrive/test_assigment')

In [ ]:
georges_path = general_path /'georges.csv'

In [ ]:
non_georges_path =general_path /'non_georges.csv'

Чтение данных и создание выброк  Мы имеем ограниченное время и возьмем часть фалов

In [ ]:
# Читаем CSV-файлы
georges_links = pd.read_csv(georges_path, header=None)[0].tolist()
not_georges_links = pd.read_csv(non_georges_path, header=None)[0].tolist()

In [ ]:
# Возьмем по 2000 примеров
georges_links = georges_links[:2000]
not_georges_links = not_georges_links[:2000]


Создание выборок
Первый шаг: Разделили данные на тренировочную выборку и временную выборку (temp_georges) в соотношении 70% / 30%
Второй шаг: Разделили временную выборку (temp_georges) на валидационную и тестовую выборки в соотношении 50% / 50%

In [ ]:
train_georges, temp_georges = train_test_split(georges_links, test_size=0.3, random_state=42)
val_georges, test_georges = train_test_split(temp_georges, test_size=0.5, random_state=42)

train_not_georges, temp_not_georges = train_test_split(not_georges_links, test_size=0.3, random_state=42)
val_not_georges, test_not_georges = train_test_split(temp_not_georges, test_size=0.5, random_state=42)

In [ ]:
# Функция для скачивания изображений
def download_images(links, folder):
    for link in links:
        try:
            filename = wget.download(link, out=folder)
        except Exception as e:
            print(f"Ошибка при скачивании {link}: {e}")

# Создаем папки
os.makedirs('/content/data/train/georges', exist_ok=True)
os.makedirs('/content/data/train/not_georges', exist_ok=True)
os.makedirs('/content/data/val/georges', exist_ok=True)
os.makedirs('/content/data/val/not_georges', exist_ok=True)
os.makedirs('/content/data/test/georges', exist_ok=True)
os.makedirs('/content/data/test/not_georges', exist_ok=True)

# Скачиваем изображения
download_images(train_georges, '/content/data/train/georges')
download_images(val_georges, '/content/data/val/georges')
download_images(test_georges, '/content/data/test/georges')

download_images(train_not_georges, '/content/data/train/not_georges')
download_images(val_not_georges, '/content/data/val/not_georges')
download_images(test_not_georges, '/content/data/test/not_georges')

Проверка изображений

In [ ]:
def check_images(folder_path):
    """
    Проверяет изображения в папке на повреждения.
    Args:
        folder_path (str): Путь к папке с изображениями.
    Returns:
        dict: Отчет о проверке (количество файлов, поврежденных файлов).
    """
    total_files = 0
    corrupted_files = 0

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        total_files += 1

        try:
            # Попытка открыть изображение
            img = Image.open(file_path)
            img.verify()  # Проверка целостности файла
            img.close()

        except Exception as e:
            print(f"Поврежденный файл {file_path}: {e}")
            corrupted_files += 1
            # Удаляем поврежденный файл
            os.remove(file_path)

    print(f"Проверено файлов: {total_files}, Поврежденных файлов: {corrupted_files}")
    return {"total_files": total_files, "corrupted_files": corrupted_files}

#Применение для всех папок
folders_to_check = [
    '/content/data/train/georges',
    '/content/data/train/not_georges',
    '/content/data/val/georges',
    '/content/data/val/not_georges'
]

for folder in folders_to_check:
    print(f"\nПроверка изображений в папке: {folder}")
    report = check_images(folder)


Проверка изображений в папке: /content/data/train/georges
Проверено файлов: 1400, Поврежденных файлов: 0

Проверка изображений в папке: /content/data/train/not_georges
Проверено файлов: 1400, Поврежденных файлов: 0

Проверка изображений в папке: /content/data/val/georges
Проверено файлов: 300, Поврежденных файлов: 0

Проверка изображений в папке: /content/data/val/not_georges
Проверено файлов: 300, Поврежденных файлов: 0


In [ ]:
def validate_images(image_paths, labels):
    """
    Проверяет все изображения на корректность: существование, размер, формат, нормализацию.
    Args:
        image_paths (list): Список путей к изображениям.
        labels (list): Список меток классов.
    Returns:
        dict: Отчет о проверке (количество файлов, поврежденных файлов, ошибок).
    """
    total_files = len(image_paths)
    corrupted_files = 0
    size_errors = 0
    normalization_errors = 0

    for path, label in zip(image_paths, labels):
        try:
            # Чтение файла
            image = tf.io.read_file(path)
            image = tf.image.decode_jpeg(image, channels=3)  # Декодирование JPEG

            # Проверка размера
            if image.shape[:2] != (128, 128):
                image = tf.image.resize(image, [128, 128])
                size_errors += 1

            # Нормализация
            image /= 255.0
            if tf.reduce_min(image).numpy() < 0 or tf.reduce_max(image).numpy() > 1:
                normalization_errors += 1

        except Exception as e:
            print(f"Ошибка при проверке файла {path}: {e}")
            corrupted_files += 1

    print(f"Проверено файлов: {total_files}")
    print(f"Поврежденных файлов: {corrupted_files}")
    print(f"Файлов с некорректным размером: {size_errors}")
    print(f"Файлов с ошибками нормализации: {normalization_errors}")

    return {
        "total_files": total_files,
        "corrupted_files": corrupted_files,
        "size_errors": size_errors,
        "normalization_errors": normalization_errors
    }

создание Tensorflow- совместимого датасета с использованием tf.data
Приведениек стандартному размеру

In [ ]:
def load_and_preprocess_image(path, label):
    """
    Загружает и предобрабатывает изображение: чтение, изменение размера и нормализация.
    Args:
        path (str): Путь к файлу изображения.
        label (int): Метка класса (1 - Георгий, 0 - не Георгий).
    Returns:
        tuple: Кортеж (изображение, метка), где изображение нормализовано и имеет размер 128x128.
    """
    image = tf.io.read_file(path)  # Чтение файла изображения
    image = tf.image.decode_jpeg(image, channels=3)  # Декодирование JPEG в RGB
    image = tf.image.resize(image, [128, 128])  # Изменение размера до 128x128
    image /= 255.0  # Нормализация значений пикселей в диапазон [0, 1]
    return image, label


def create_dataset(image_paths, labels, batch_size=64):
    """
    Создает TensorFlow-совместимый датасет из путей к изображениям и меток.
    Args:
        image_paths (list): Список путей к изображениям.
        labels (list): Список меток классов.
        batch_size (int): Размер батча.
    Returns:
        tf.data.Dataset: Оптимизированный датасет с перемешиванием, батчами и предзагрузкой.
    """
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))  # Создание датасета
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)  # Применение предобработки
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)  # Оптимизация
    return dataset


# Пути к изображениям
train_paths = [
    '/content/data/train/georges/' + f for f in os.listdir('/content/data/train/georges')
] + [
    '/content/data/train/not_georges/' + f for f in os.listdir('/content/data/train/not_georges')
]
train_labels = [1] * len(os.listdir('/content/data/train/georges')) + \
               [0] * len(os.listdir('/content/data/train/not_georges'))
"""
Создает списки путей и меток для тренировочной выборки.
"""

val_paths = [
    '/content/data/val/georges/' + f for f in os.listdir('/content/data/val/georges')
] + [
    '/content/data/val/not_georges/' + f for f in os.listdir('/content/data/val/not_georges')
]
val_labels = [1] * len(os.listdir('/content/data/val/georges')) + \
             [0] * len(os.listdir('/content/data/val/not_georges'))
"""
Создает списки путей и меток для валидационной выборки.
"""

# Создаем датасеты
train_dataset = create_dataset(train_paths, train_labels)
val_dataset = create_dataset(val_paths, val_labels)

# Отчет о создании датасетов
print("\n=== Отчет о создании датасетов ===")
print(f"Тренировочный датасет создан: {len(train_paths)} изображений приведены к размеру 128x128 и нормализованы.")
print(f"Валидационный датасет создан: {len(val_paths)} изображений приведены к размеру 128x128 и нормализованы.")


=== Отчет о создании датасетов ===
Тренировочный датасет создан: 2800 изображений приведены к размеру 128x128 и нормализованы.
Валидационный датасет создан: 600 изображений приведены к размеру 128x128 и нормализованы.


In [ ]:
# Проверка тренировочных данных
print("\n=== Проверка тренировочных данных ===")
train_report = validate_images(train_paths, train_labels)

# Проверка валидационных данных
print("\n=== Проверка валидационных данных ===")
val_report = validate_images(val_paths, val_labels)

# Вывод итогового отчета
print("\n=== Итоговый отчет ===")
print("Тренировочные данные:")
print(f"  Всего файлов: {train_report['total_files']}")
print(f"  Поврежденных файлов: {train_report['corrupted_files']}")
print(f"  Файлов с некорректным размером: {train_report['size_errors']}")
print(f"  Файлов с ошибками нормализации: {train_report['normalization_errors']}")

print("\nВалидационные данные:")
print(f"  Всего файлов: {val_report['total_files']}")
print(f"  Поврежденных файлов: {val_report['corrupted_files']}")
print(f"  Файлов с некорректным размером: {val_report['size_errors']}")
print(f"  Файлов с ошибками нормализации: {val_report['normalization_errors']}")



=== Проверка тренировочных данных ===
Проверено файлов: 2800
Поврежденных файлов: 0
Файлов с некорректным размером: 2800
Файлов с ошибками нормализации: 0

=== Проверка валидационных данных ===
Проверено файлов: 600
Поврежденных файлов: 0
Файлов с некорректным размером: 600
Файлов с ошибками нормализации: 0

=== Итоговый отчет ===
Тренировочные данные:
  Всего файлов: 2800
  Поврежденных файлов: 0
  Файлов с некорректным размером: 2800
  Файлов с ошибками нормализации: 0

Валидационные данные:
  Всего файлов: 600
  Поврежденных файлов: 0
  Файлов с некорректным размером: 600
  Файлов с ошибками нормализации: 0


Загружаем предобученную модель

In [ ]:
def create_resnet_model(input_shape=(128, 128, 3), num_classes=1):
    """
    Создает модель на основе ResNet50 для бинарной классификации.
    Args:
        input_shape (tuple): Размер входных изображений (высота, ширина, каналы).
        num_classes (int): Количество классов (1 для бинарной классификации).
    Returns:
        tf.keras.Model: Скомпилированная модель.
    """
    # Загрузка предобученной модели ResNet50 с замороженными слоями
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Замораживаем слои ResNet50

    # Создание новой модели поверх ResNet50
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),  # Уменьшаем размерность выхода ResNet50
        Dense(128, activation='relu'),  # Полносвязный слой
        Dense(num_classes, activation='sigmoid')  # Выходной слой для бинарной классификации
    ])

    # Компиляция модели
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

Создани и обучение модели

In [ ]:
# Создаем модель
model = create_resnet_model()

# Выводим сводку модели
model.summary()

# Обучение модели
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,  # Начнем с 30 эпох
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]  # ранняя остановка
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 4, 4, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 42s 571ms/step - accuracy: 0.7704 - loss: 0.6613 - val_accuracy: 0.5000 - val_loss: 1.0821
Epoch 2/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 328ms/step - accuracy: 0.2348 - loss: 1.2231 - val_accuracy: 0.5000 - val_loss: 0.7685
Epoch 3/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 21s 348ms/step - accuracy: 0.2032 - loss: 0.9427 - val_accuracy: 0.5000 - val_loss: 0.7241
Epoch 4/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 327ms/step - accuracy: 0.2017 - loss: 0.8560 - val_accuracy: 0.5000 - val_loss: 0.7187
Epoch 5/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 22s 330ms/step - accuracy: 0.2025 - loss: 0.8390 - val_accuracy: 0.5000 - val_loss: 0.7044
Epoch 6/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 333ms/step - accuracy: 0.2038 - loss: 0.8060 - val_accuracy: 0.5000 - val_loss: 0.7034
Epoch 7/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 21s 350ms/step - accuracy: 0.2066 - loss: 0.7991 - val_accuracy: 0.5000 - val_loss: 0.6978
Epoch 8/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 331ms/step - accuracy: 0.2032 - loss: 0.7827 - val_accu

Разморозка дополнительных  слоев и  дообучение

In [ ]:
model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
history_fine = model.fit(train_dataset, validation_data=val_dataset, epochs=20)

Epoch 1/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 38s 525ms/step - accuracy: 0.3661 - loss: 0.7022 - val_accuracy: 0.5733 - val_loss: 0.6805
Epoch 2/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 20s 327ms/step - accuracy: 0.3736 - loss: 0.7018 - val_accuracy: 0.5750 - val_loss: 0.6803
Epoch 3/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 21s 370ms/step - accuracy: 0.3699 - loss: 0.7017 - val_accuracy: 0.5750 - val_loss: 0.6800
Epoch 4/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 39s 329ms/step - accuracy: 0.3822 - loss: 0.7002 - val_accuracy: 0.5817 - val_loss: 0.6797
Epoch 5/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 21s 347ms/step - accuracy: 0.3862 - loss: 0.7001 - val_accuracy: 0.5817 - val_loss: 0.6795
Epoch 6/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 330ms/step - accuracy: 0.3649 - loss: 0.7019 - val_accuracy: 0.5800 - val_loss: 0.6792
Epoch 7/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 20s 315ms/step - accuracy: 0.3803 - loss: 0.6996 - val_accuracy: 0.5750 - val_loss: 0.6789
Epoch 8/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 330ms/step - accuracy: 0.3944 - loss: 0.6976 - val_accu

Продолжим обучение

In [ ]:
model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
history_fine = model.fit(train_dataset, validation_data=val_dataset, epochs=50)

Epoch 1/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 38s 528ms/step - accuracy: 0.4528 - loss: 0.6891 - val_accuracy: 0.5950 - val_loss: 0.6752
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 29s 332ms/step - accuracy: 0.4750 - loss: 0.6866 - val_accuracy: 0.5933 - val_loss: 0.6751
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 21s 332ms/step - accuracy: 0.4695 - loss: 0.6875 - val_accuracy: 0.5967 - val_loss: 0.6750
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 21s 331ms/step - accuracy: 0.4621 - loss: 0.6880 - val_accuracy: 0.5950 - val_loss: 0.6748
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 20s 330ms/step - accuracy: 0.4675 - loss: 0.6868 - val_accuracy: 0.5933 - val_loss: 0.6747
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 22s 331ms/step - accuracy: 0.4641 - loss: 0.6862 - val_accuracy: 0.5950 - val_loss: 0.6745
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 333ms/step - accuracy: 0.4524 - loss: 0.6893 - val_accuracy: 0.5967 - val_loss: 0.6744
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 22s 327ms/step - accuracy: 0.4608 - loss: 0.6874 - val_accu

In [ ]:
# Обучение модели с ранней остановкой
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Перекомпиляция модели с меньшей скоростью обучения
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Дообучение модели
history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[early_stopping]
)